In [34]:
import pandas as pd
import numpy as np
from datetime import datetime
import preprocessing as p

In [6]:
df1 = pd.read_csv("sample/2014_01_01.csv")
df2 = pd.read_csv("sample/2014_01_02.csv")
df3 = pd.read_csv("sample/2014_01_03.csv")

In [7]:
p.to_sec(df1)
p.to_sec(df2)
p.to_sec(df3)
p.to_ID(df1)
p.to_ID(df2)
p.to_ID(df3)

## ID-agnostic mode

In [20]:
l = []
l += df1[df1.ID == 1]['sec'].tolist()
l += df2[df2.ID == 1]['sec'].tolist()
l += df3[df3.ID == 1]['sec'].tolist()

In [28]:
l.sort()

In [57]:
DIM = 3
SIGMA = 5000

In [43]:
n_tuples = []
for i in range(len(l)-DIM):
    n_tuples.append(np.array(l[i:i+DIM]))

In [44]:
Tr = n_tuples[-1]
#편의상 하루 중 가장 마지막 기록으로 설정함.
#추후에 실시간으로 돌릴때에는 최근값으로 다시 설정해야함

In [58]:
predicted = []
for i,val in enumerate(n_tuples[:-1]):
    #maximum norm
    if np.linalg.norm(Tr-val, ord = np.inf) < SIGMA:
        predicted.append(n_tuples[i+1][0])

In [59]:
predicted

[81021.0, 81021.0, 81259.0, 82808.0, 82811.0, 83035.0]

일별 초단위로 계산: 특정 시간대에 대한 특징 반영 가능. 대신 다음날로 넘어가는 부분의 연속성은 파악하기 어렵다

## ID-aware mode
To be continued..